In [2]:
import os
import glob
import numpy as np
from os.path import join
from medpy.io import load
from jarvis.utils.display import imshow
data_root = 'D:\\data/'
# data_root = '/home/mramados/data'
os.makedirs('./data', exist_ok=True)

In [3]:
orca_data_train = join(data_root, 'orca', 'Training_Set', 'Train')
orca_data_valid = join(data_root, 'orca', 'Training_Set', 'Validation')
orca_data_test = join(data_root, 'orca', 'Testing_Set')

orca_train_paths = [join(orca_data_train, secondary_path) 
    for secondary_path in os.listdir(orca_data_train)]
orca_valid_paths = [join(orca_data_valid, secondary_path) 
    for secondary_path in os.listdir(orca_data_valid)]
orca_test_paths =  [join(orca_data_test, secondary_path) 
    for secondary_path in os.listdir(orca_data_valid)]
    
orca_data = None
orca_label = None

for _path in orca_train_paths:
    misc_name = glob.glob1(_path, '*ctai.mhd')[0]
    data_name = glob.glob1(_path, '*cti.mhd')[0]
    label_name = glob.glob1(_path, '*r.mhd')[0]
    (data, _), (label, label_meta) = load(join(_path, data_name)), load(join(_path, label_name))
    data = np.transpose(np.rot90(data, 3), (2, 0, 1))
    label = np.transpose(np.rot90(label, 3), (2, 0, 1))
    data = np.clip(data, 0, 64) / 32
    label = np.clip(label, 0, 1)
    if orca_data is None:
        orca_data = data
        orca_label = label
    else:
        orca_data = np.concatenate([orca_data, data], axis=0)
        orca_label = np.concatenate([orca_label, label], axis=0)
print(orca_data.shape, orca_label.shape)       
orca_data = np.asarray(orca_data)
np.save('./data/orca_data.npy', orca_data)
orca_label = np.asarray(orca_label)
np.save('./data/orca_label.npy', orca_label)

  0%|          | 0/32 [00:00<?, ?it/s]

(1540, 512, 512) (1540, 512, 512)


In [6]:
_data = join(data_root, 'structseg')
structseg_train_paths = [join(_data, secondary_path) 
    for secondary_path in os.listdir(_data)]

from scipy import ndimage
crop = 512
zoom = 2.5
center_x = 20
center_y = 10
structseg_data = None
structseg_label = None

for _path in structseg_train_paths:
    data_name = glob.glob1(_path, 'data.nii.gz')[0]
    label_name = glob.glob1(_path, 'label.nii.gz')[0]
    (data, _), (label, label_meta) = load(join(_path, data_name)), load(join(_path, label_name))
    data = np.transpose(np.rot90(data, 3), (2, 0, 1))
    label = np.transpose(np.rot90(label, 3), (2, 0, 1))
    data = np.flip(data, axis=(2))
    label = np.flip(label, axis=(2))
    min = np.min(data)
    data = ndimage.interpolation.zoom(data, (1, zoom, zoom), order=3)
    label = ndimage.interpolation.zoom(label, (1, zoom, zoom), order=0)
    shape = data.shape[-2]    
    start = shape // 2 - crop // 2    
    t1 = start - center_x
    data = data[:, start - center_x:start + crop - center_x, start - center_y:start + crop - center_y]
    label = label[:, start - center_x:start + crop - center_x, start - center_y:start + crop - center_y]
    lx, ly = data.shape[-2], data.shape[-1]
    X, Y = np.ogrid[0:lx, 0:ly]
    mask = (X - lx / 2) ** 2 + (Y - ly / 2) ** 2 > lx * ly / 4
    data[:, mask] = min
    label[:, mask] = 0        
    data = np.clip(data, 0, 64) / 32
    label = np.clip(label==3, 0, 1)
    if structseg_data is None:
        structseg_data = data
        structseg_label = label       
    else:
        structseg_data = np.concatenate([structseg_data, data], axis=0)
        structseg_label = np.concatenate([structseg_label, label], axis=0)
print(structseg_data.shape, structseg_label.shape)        
structseg_data = np.asarray(structseg_data)
np.save('./data/structseg_data.npy', structseg_data)
structseg_label = np.asarray(structseg_label)
np.save('./data/structseg_label.npy', structseg_label)

(4775, 512, 512) (4775, 512, 512)


TypeError: _save_dispatcher() missing 1 required positional argument: 'arr'

In [8]:
_data = join(data_root, 'plaque')
plaque_train_paths = [join(_data, secondary_path) 
    for secondary_path in os.listdir(_data)]
plaque_data = None
plaque_label = None
for _path in plaque_train_paths:
    data_name = glob.glob1(_path, '*cti.npy')[0]
    label_name = glob.glob1(_path, '*r.npy')[0]
    data, label = np.load(join(_path, data_name)), np.load(join(_path, label_name))
    #data = np.transpose(np.rot90(data, 3), (2, 0, 1))
    #label = np.transpose(np.rot90(label, 3), (2, 0, 1))
    data = np.clip(data, 0, 64) / 32
    label = np.clip(label, 0, 1)
    if plaque_data is None:
        plaque_data = data
        plaque_label = label
    else:
        plaque_data = np.concatenate([plaque_data, data], axis=0)
        plaque_label = np.concatenate([plaque_label, label], axis=0)
print(plaque_data.shape, plaque_label.shape)
plaque_data = np.asarray(plaque_data)
np.save('./data/plaque_data.npy', plaque_data)
plaque_label = np.asarray(plaque_label)
np.save('./data/plaque_label.npy', plaque_label)

(1540, 512, 512) (1540, 512, 512)


In [1]:
!python struct_seg.py

^C


In [ ]:
!python plaque_seg.py